In [1]:
import numpy as np 
import pandas as pd
import networkx as nx

from utils import plot_cluster, graph_sum
from make_synthetic_data import generate_synthetic_data_with_hierarchy
from make_synthetic_data import create_synthetic_paritally_known_label
from graph import knn_graph, partition
from chameleon_cluster import partition_phase, merge_phase

## Generate synthetic data

In [2]:
data = generate_synthetic_data_with_hierarchy()

In [3]:
data = create_synthetic_paritally_known_label(data)

In [4]:
plot_cluster(data, cluster_col='true_clst_l3')

### Cannot-link constraints

Create cannot-link matrix based on known tags. If two instances have known different tags, they will be marked as cannot-link.  
The cannot-link matrix will be input in merge phase to avoid merging clusters that has cannot-link instances.

In [5]:
# a matrix showing whether two instances has the same tag at a certain level
same_l3 = data['known_tag_l3'].values.reshape(-1, 1) == data['known_tag_l3'].values.reshape(1, -1)
same_l2 = data['known_tag_l2'].values.reshape(-1, 1) == data['known_tag_l2'].values.reshape(1, -1)
same_l1 = data['known_tag_l1'].values.reshape(-1, 1) == data['known_tag_l1'].values.reshape(1, -1)

# cannot-link matrix: known tag is not null and known tag is different
valid_l3 = data['known_tag_l3'].notna().values
valid_l2 = data['known_tag_l2'].notna().values
valid_l1 = data['known_tag_l1'].notna().values
cl_l3 = ~same_l3 & valid_l3.reshape(-1, 1) & valid_l3.reshape(1, -1)
cl_l2 = ~same_l2 & valid_l2.reshape(-1, 1) & valid_l2.reshape(1, -1)
cl_l1 = ~same_l1 & valid_l1.reshape(-1, 1) & valid_l1.reshape(1, -1)
cl_mat = cl_l3 | cl_l2 | cl_l1

In [6]:
print(cl_mat.shape)
cl_mat

(10000, 10000)


array([[False,  True, False, ...,  True, False, False],
       [ True, False, False, ...,  True,  True, False],
       [False, False, False, ..., False, False, False],
       ...,
       [ True,  True, False, ..., False,  True, False],
       [False,  True, False, ...,  True, False, False],
       [False, False, False, ..., False, False, False]])

## Generate graph from data

In [7]:
graph = knn_graph(data.iloc[:, :5])

In [8]:
plot_cluster(data, graph)

## Pre-label cluster according to partially known label

Prelabel cluster according to partially known label.  
If a instance's cluster is known at the finest level (l3 in this case), then these cluster are excluded in the partition phase, i.e., they won't be cut into smaller subclusters.

In [9]:
# pre partition according to the known tag of the finest level
known_tag_l3_dict = data['known_tag_l3'].dropna().to_dict()
nx.set_node_attributes(graph, known_tag_l3_dict, 'cluster')
exclude_cluster = np.unique(list(known_tag_l3_dict.values())).tolist()

cluster_idxs = pd.DataFrame({'cluster': pd.Series(nx.get_node_attributes(graph, 'cluster'))})
labeled_nodes = list(cluster_idxs.index)
known_tag_l2_dict = data['known_tag_l2'].drop(labeled_nodes).dropna().to_dict()
nx.set_node_attributes(graph, known_tag_l2_dict, 'cluster')

cluster_idxs = pd.DataFrame({'cluster': pd.Series(nx.get_node_attributes(graph, 'cluster'))})
labeled_nodes = list(cluster_idxs.index)
known_tag_l1_dict = data['known_tag_l1'].drop(labeled_nodes).dropna().to_dict()
nx.set_node_attributes(graph, known_tag_l1_dict, 'cluster')

In [10]:
exclude_cluster

['0', '1', '2', '3', '4', '5']

In [11]:
graph_sum(graph)

,count
cluster,
"(0, 1)",324
"(0, 1, 2, 3)",1381
"(2, 3)",352
"(4, 5)",1024
0,163
1,149
2,163
3,192
4,158


In [12]:
plot_cluster(data, graph)

## Partition Phase

In [13]:
partition_phase(graph, n_cluster_final=30, exclude_cluster=exclude_cluster)

Eixst nodes without cluster. Initialize 5956 nodes to cluster -1
Number of clusters: 12
Number of clusters: 13
Number of clusters: 14
Number of clusters: 15
Number of clusters: 16
Number of clusters: 17
Number of clusters: 18
Number of clusters: 19
Number of clusters: 20
Number of clusters: 21
Number of clusters: 22
Number of clusters: 23
Number of clusters: 24
Number of clusters: 25
Number of clusters: 26
Number of clusters: 27
Number of clusters: 28
Number of clusters: 29
Number of clusters: 30


In [14]:
plot_cluster(data, graph)

In [15]:
graph_sum(graph)

,count
cluster,
-1,374
0,373
1,372
2,372
3,373
4,373
5,372
6,372
7,346


## Merge Phase

In [16]:
merge_phase(graph, n_cluster_final=6, cl_mat=cl_mat)

Output()

Number of clusters: 29, Merged pairs: ('(0, 1)', '1')

Number of clusters: 28, Merged pairs: ('(2, 3)', '2')

Number of clusters: 27, Merged pairs: (17, '(0, 1)')

Number of clusters: 26, Merged pairs: ('(2, 3)', 7)

...

Number of clusters: 10, Merged pairs: (1, 4)

Number of clusters: 9, Merged pairs: (6, '3')

Number of clusters: 8, Merged pairs: ('(4, 5)', 0)

Number of clusters: 7, Merged pairs: (15, 9)

Number of clusters: 6, Merged pairs: (2, 5)

In [17]:
plot_cluster(data, graph)